In [1]:
import pandas as pd
import numpy as np
import simpy
import random

class Sala:
    def __init__(self, env, id, num_pacientes):
        self.env = env
        self.id = id
        self.num_pacientes = simpy.Container(env, init=num_pacientes, capacity=num_pacientes)

class Clinica:
    def __init__(self, env, salas, doctores, cabinas_auto):
        self.env = env
        self.salas = salas
        self.doctores = simpy.Resource(env, capacity=doctores)
        self.cabinas_auto = simpy.Resource(env, capacity=cabinas_auto)

class Paciente:
    def __init__(self, env, id, sala, necesidad_consulta, clinica):
        self.env = env
        self.id = id
        self.sala: Sala = sala
        self.necesidad_consulta = necesidad_consulta
        self.consulta_automatica = random.choice([True, False])
        self.clinica: Clinica = clinica
        self.recibio_atencion = False
        self.env.process(self.start())
        self.eventos = pd.DataFrame()

    def start(self):
        print(f"{self.env.now}: Paciente {self.id} (auto: {self.consulta_automatica}) llegó con necesidad de {self.necesidad_consulta} consultas")
        self.eventos = pd.concat([self.eventos, pd.DataFrame({
            "id": [self.id], "evento": ["LLEGO"], "time": [self.env.now]
        })])

        limite_inicial = 5 if self.consulta_automatica else 8
        limite_superior = 10 if self.consulta_automatica else 15
        recurso = self.clinica.cabinas_auto if self.consulta_automatica else self.clinica.doctores

        with recurso.request() as request:
            yield request
            print(f"{self.env.now}: Paciente {self.id} siendo atendido por {'cabina auto' if self.consulta_automatica else 'doctor'}")
            self.eventos = pd.concat([self.eventos, pd.DataFrame({
                "id": [self.id], "evento": ["ATENDIDO"], "time": [self.env.now]
            })])

            if self.sala.num_pacientes.level > self.necesidad_consulta:
                yield self.sala.num_pacientes.get(self.necesidad_consulta)
                yield self.env.timeout(random.randint(limite_inicial, limite_superior))
                self.recibio_atencion = True
                print(f"{self.env.now}: Paciente {self.id} recibió atención")
                self.eventos = pd.concat([self.eventos, pd.DataFrame({
                    "id": [self.id], "evento": ["ATENDIDO EXITOSO"], "time": [self.env.now]
                })])
            else:
                print(f"{self.env.now}: Paciente {self.id} no fue atendido por falta de espacio")
                self.eventos = pd.concat([self.eventos, pd.DataFrame({
                    "id": [self.id], "evento": ["SIN ATENCION"], "time": [self.env.now]
                })])

class Simulacion:
    def __init__(self, env, salas_settings, num_doctores, num_cabinas_auto):
        self.env = env
        self.salas = [Sala(env, s["id"], s["capacidad"]) for s in salas_settings]
        self.clinica = Clinica(env, self.salas, num_doctores, num_cabinas_auto)
        self.pacientes = []
        self.env.process(self.start())

    def start(self):
        while True:
            yield self.env.timeout(random.randint(1, 5))
            for _ in range(random.randint(1, 7)):
                paciente = Paciente(
                    self.env,
                    len(self.pacientes) + 1,
                    random.choice(self.salas),
                    random.randint(1, 4),
                    self.clinica
                )
                self.pacientes.append(paciente)

# Iniciar simulación
env = simpy.Environment()
salas_settings = [
    {"id": 1, "capacidad": 100},
    {"id": 2, "capacidad": 100},
    {"id": 3, "capacidad": 100},
]

num_doctores = 5
num_cabinas_auto = 3

random.seed(111)
simulacion = Simulacion(env, salas_settings, num_doctores, num_cabinas_auto)

env.run(until=60 * 8)  # Simulación por 8 horas

# Imprimir niveles restantes en cada sala
for s in simulacion.clinica.salas:
    print(f"Sala {s.id}: {s.num_pacientes.level} espacios restantes")


2: Paciente 1 (auto: False) llegó con necesidad de 2 consultas
2: Paciente 2 (auto: True) llegó con necesidad de 2 consultas
2: Paciente 3 (auto: True) llegó con necesidad de 4 consultas
2: Paciente 1 siendo atendido por doctor
2: Paciente 2 siendo atendido por cabina auto
2: Paciente 3 siendo atendido por cabina auto
4: Paciente 4 (auto: True) llegó con necesidad de 4 consultas
4: Paciente 5 (auto: False) llegó con necesidad de 3 consultas
4: Paciente 6 (auto: True) llegó con necesidad de 3 consultas
4: Paciente 7 (auto: False) llegó con necesidad de 4 consultas
4: Paciente 8 (auto: True) llegó con necesidad de 4 consultas
4: Paciente 9 (auto: False) llegó con necesidad de 1 consultas
4: Paciente 4 siendo atendido por cabina auto
4: Paciente 5 siendo atendido por doctor
4: Paciente 7 siendo atendido por doctor
4: Paciente 9 siendo atendido por doctor
8: Paciente 10 (auto: False) llegó con necesidad de 4 consultas
8: Paciente 11 (auto: False) llegó con necesidad de 4 consultas
8: Pacie